In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
from torch.autograd import Variable
import matplotlib.pyplot as plt
#import seaborn
#seaborn.set_context(context="talk")
%matplotlib inline

[The Annotated Transformer](https://nlp.seas.harvard.edu/2018/04/03/attention.html)를 해석함.

In [2]:
def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

## Attention
어텐션 함수는 쿼리(query)와 일련의 키-값(key-value)쌍을 출력으로 매핑하는 함수로   
설명될 수 있으며, 여기서, 쿼리, 키, 값, 출력은 모두 벡터입니다. 출력은 값들의 가중합   
(weighted sum)으로 계산되며, 각 값에 할당되는 가중치는 해당 키와 쿼리 간의 어텐션 함수  
(compatibility function)를 통해 계산됨.  

우리는 이러한 어텐션을 "Scaled Dot-Production Attention)"이라고 부릅니다.  
입력은 차원이 $d_k$인 쿼리와 키, 그리고 차원이 $d_v$인 값을 포함함. 쿼리와 모든 키 간의  
내적(dot product)을 계산한 후, 각 내적 값은 $\sqrt(d_{k})$로 나누고, 소프트맥스(softmax)  
함수를 적용하여 값에 대한 가중치를 얻음.    

![fig1](../images/the-annotated-transformer_33_0.png)

실제로는, 어텐션 함수는 여러 개의 쿼리에 대해 동시에 계산되며, 이들은 행렬 𝑄로 함께 묶어(packed)  
처리됩니다. 키와 값도 각각 행렬 𝐾와 𝑉로 묶어 처리됩니다. 출력 행렬은 다음과 같이 계산됩니다:  

$    \text{attention}(Q,K,V)=softmax(\frac{QK^\top}{\sqrt{d_k}})V $




In [3]:
#h = 8(number of multi-head)
#d_k = d_model/h
#query.shape = [n_batch, h, seq_len, d_k]
#key.shape   = [n_batch, h, seq_len, d_k]
#value.shape = [n_batch, h, seq_len, d_k]
def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

가장 널리 사용되는 두 가지 어텐션 함수는 가산 어텐션과 내적 어텐션입니다.  내적 어텐션은 우리가   
제안한 알고리즘과 동일하지만, 차이점은 스케일링 계수 $\frac{1}{\sqrt{d_k}}$의 유무입니다.   

가산 어텐션은 쿼리와 키의 적합도은 은닉층 하나로 구성된 피드포워드 신경망을 계산됨.  
이 두 방식은 이론적인 계산 복잡도는 비슷하지만, 내적 어텐션은 고도로 최적화된 행렬 곱셉 코드를  
사용할 수 있기 때문에, 실제로는 훨씬 빠르고 메모리 효율이 높습니다.  

쿼리와 키의 차원 수 $d_k$가 작을 경우에는 두 메커니즘이 유사한 성능을 보이지만, $d_k$가 큰 경우에는  
가산 어텐션이 스케일링이 적용되지 않은 내적 어텐션보다 더 나은 성능을 보입니다.  
우리는 그 이유를 다음과 같이 추정합니다:  

$d_k$가 클수록 쿼리 q와 키 k의 내적이 커지고, 이는 softmax 함수의 기울기 거의 0에 가까운 영역으로  
들어가게 말들기 때문임. 예를 들어, q와 k의 각 성분이 평균이 0이고 분산 1인 독립적인 확률변수라고   
가정하면, 그 내적 $ q\cdot{k}=  \sum_{i=0}^{n} q_ik_i $ 는 평균이 '0'이고 분산이 $d_k$ 임.  
이 효과를 상쇄하기 위해, 우리는 내적에 $\frac{1}{\sqrt{d_k}}$을 곱함.


"$d_{model}$ 차원의 키(key), 값(value), 쿼리(query)를 사용해 단일 어텐션 함수를 수행하는 대신,
쿼리, 키, 값을 각각 학습된 선형 변환(linear projection)을 통해 h번 투영하여, 각각 $d_k$, $d_k$, $d_v$ 차원으로 변환하는 것이 더 효과적이라는 것을 발견했습니다.<br>
$Q=W^QQ, K=W^KK$,$V=W^VV$ <br>
이렇게 투영된 쿼리, 키, 값 각각에 대해 병렬로 어텐션 연산을 수행하면, d_v 차원의 출력 벡터들이 생성됩니다. 이 출력 벡터들은 이어서 하나로 연결(concatenate)된 후, 다시 한 번 선형 변환을 통해 최종 출력 값이 생성됩니다. (이 과정은 그림 2에 나타나 있습니다.)

![fig2](../images/the-annotated-transformer_38_0.png)  
Figure 2: Multi-Head Attention consists of several attention layers running in parallel.

**멀티-헤드 어텐션(Multi-head Attention)**은 모델이 서로 다른 위치에서  
서로 다른 표현 하위 공간(representation subspace)의 정보를 동시에(attend jointly)  
참조할 수 있도록 해줍니다. 단일 어텐션 헤드만 사용할 경우,   
평균화(averaging)가 이러한 정보를 억제하는 경향이 있습니다.

$ \text{MultiHead}(Q,K,V)=\text{Concat}(head_1,\dots,head_h)W^O $

여기서,  

$ \text{head}_i = \text{Attention}(QW_i^Q, KW_i^K, VW_i^V) $  

각 head에 대해 사용하는 프로젝션 행렬들은 다음과 같습니다:  

- $W_i^Q \in \mathbb{R}^{d_{\text{model}}\times d_k}$
- $W_i^K \in \mathbb{R}^{d_{\text{model}}\times d_k}$
- $W_i^V \in \mathbb{R}^{d_{\text{model}}\times d_v}$
- $W_i^O \in \mathbb{R}^{hd_v \times d_{model}}$

이 논문에서는 **h=8** 개의 병렬 어텐션 레이어, 즉 8개의 헤드를 사용합니다. 각 헤드에 대해  
**$d_k=d_v=d_{\text{model}}/h=64 $** 설정됨.  
각 헤드의 차원이 줄어들기 때문에, 전체 연산량은 전체 차원을 사용하는 단일 헤드 어텐션과   
유사한 수준으로 유지됨.

In [4]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, query, key, value, mask=None):
        "Implements Figure 2"
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)

        # 1) Do all the linear projections in batch from d_model => h x d_k
        query, key, value = [
            lin(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
            for lin, x in zip(self.linears, (query, key, value))
        ]

        print("query.shape : ", query.shape)
        print("key.shape   : ", key.shape)
        print("value.shape : ", value.shape)

        # 2) Apply attention on all the projected vectors in batch.
        x, self.attn = attention(
            query, key, value, mask=mask, dropout=self.dropout
        )

        print("x.shape : ", x.shape)

        # 3) "Concat" using a view and apply a final linear.
        x = (
            x.transpose(1, 2)
            .contiguous()
            .view(nbatches, -1, self.h * self.d_k)
        )
        
        del query
        del key
        del value
        return self.linears[-1](x)

```  python
query, key, value = [
    lin(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
    for lin, x in zip(self.linears, (query, key, value))
]
```

self.linears는 각 $W^Q, W^K, W^V, W^O$에 대한 행렬을 의미함.  
$W^Q$는 $W^{d_{k}}$가 8개가 concat되어있는 행렬임.  
이코드는 예를 들어 Q shape이 (n_batch, seq_len, d_model)의 크기 일때  
$Q = W^Q \times Q$ 이고 shape은 (n_batch, seq_len, d_model)이 되고  
이걸 (n_batch,  h, seq_len, $d_k$)로 변경하는 코드임.


In [5]:
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

In [6]:
n_batch,d_model,h=1, 512, 8
seq_len = 10

In [7]:
embedding = Embeddings(d_model,11)

In [8]:
x = torch.from_numpy(np.random.randint(0, 11, size=(1, 10)))
xx = embedding(x)
print("xx input shape : ", xx.shape)

xx input shape :  torch.Size([1, 10, 512])


In [9]:
attn = MultiHeadedAttention(h, d_model)
out = attn(xx,xx,xx)
print("out shape : ", out.shape)
print("attn shape : ", attn.attn.shape)


query.shape :  torch.Size([1, 8, 10, 64])
key.shape   :  torch.Size([1, 8, 10, 64])
value.shape :  torch.Size([1, 8, 10, 64])
x.shape :  torch.Size([1, 8, 10, 64])
out shape :  torch.Size([1, 10, 512])
attn shape :  torch.Size([1, 8, 10, 10])
